In [1]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import gc
import warnings
from tqdm import tqdm
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold
from itertools import product
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.preprocessing import LabelEncoder
from scipy import stats
from feature_selector import FeatureSelector

warnings.filterwarnings('ignore')

pd.set_option('display.unicode.ambiguous_as_wide', True)
pd.set_option('display.unicode.east_asian_width', True)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option("display.max_colwidth", 100)
pd.set_option('display.width', 1000)

In [2]:
# 节省内存读文件
def reduce_mem_usage(df):
    """
    iterate through all the columns of a dataframe and modify the data type to reduce memory usage.
    @param df:
    @return:
    """
    start_mem = df.memory_usage().sum()
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('str')

    end_mem = df.memory_usage().sum()
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
def count_encode(df, cols):
    for col in tqdm(cols):
        print(col)
        vc = df[col].value_counts(dropna=True, normalize=True)
        df[col + '_count'] = df[col].map(vc).astype('float32')
    return df
        
        
def label_encode(df, cols):
    le = LabelEncoder()
    for col in tqdm(cols):
        df[col] = df[col].fillna('NA')
        df[col] = le.fit_transform(df[col].astype(str))
    return df


def cross_cat_num(df, cat_col, num_col):
    for f1 in tqdm(cat_col):
        g = df.groupby(f1, as_index=False)
        for f2 in tqdm(num_col):
            df_new = g[f2].agg({
                '{}_{}_max'.format(f1, f2): 'max',
                '{}_{}_min'.format(f1, f2): 'min',
                '{}_{}_median'.format(f1, f2): 'median',
                '{}_{}_mean'.format(f1, f2): 'mean',
                '{}_{}_skew'.format(f1, f2): 'skew',
                '{}_{}_nunique'.format(f1, f2): 'nunique'
            })
            df = df.merge(df_new, on=f1, how='left')
            del df_new
            gc.collect()
    return df

## user表

In [4]:
train_user = pd.read_hdf('../input/train_user.h5')
test_user = pd.read_hdf('../input/test_user.h5')

In [5]:
# 将city_name和county_name拼接起来
train_user['city_name_county_name'] = train_user['city_name'].astype(str) + '_' + train_user['county_name'].astype(str)
test_user['city_name_county_name'] = test_user['city_name'].astype(str) + '_' + test_user['county_name'].astype(str)

In [6]:
cat_feat = ['city_name', 'county_name', 'city_name_county_name']
num_feat = ['idcard_cnt', 'arpu_202004', 'idcard_cnt*arpu_202004', 'arpu_202004/idcard_cnt']

In [7]:
# y = train_user['label']

In [8]:
# ME = MeanEncoder(categorical_features=cat_feat,
#                  n_splits=3,
#                  target_type='classification',
#                  prior_weight_func=None)
# X_data = ME.fit_transform(train_user.drop('label', axis=1), y)
# X_test = ME.transform(test_user)

In [9]:
# train_user = X_data.copy()
# train_user['label'] = y
# test_user = X_test.copy()

# train_user.shape, test_user.shape

In [10]:
df_user = pd.concat([train_user, test_user])

del train_user, test_user
gc.collect()

0

In [11]:
# 电话的数量*月消费额
df_user['idcard_cnt*arpu_202004'] = df_user['idcard_cnt'] * df_user['arpu_202004']

# 月消费额/电话的数量
df_user['arpu_202004/idcard_cnt'] = df_user['arpu_202004'] / (df_user['idcard_cnt'] + 0.0001)

In [12]:
df_user = cross_cat_num(df_user, cat_feat, num_feat)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  3.73it/s]


In [13]:
df_user = count_encode(df_user, cat_feat + ['idcard_cnt'])

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 308.51it/s]

city_name
county_name
city_name_county_name
idcard_cnt


In [14]:
df_user = label_encode(df_user, cat_feat)

100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 214.85it/s]


In [15]:
df_user.to_hdf('../input/user_features.h5', 'df', index=False)

del df_user
gc.collect()

66

## voc表

In [16]:
train_voc = pd.read_hdf('../input/train_voc.h5')
test_voc = pd.read_hdf('../input/test_voc.h5')

In [17]:
df_voc = pd.concat([train_voc, test_voc])

del train_voc, test_voc
gc.collect()

128

In [18]:
df_voc['city_name_county_name'] = df_voc['city_name'] + '_' + df_voc['county_name']

In [19]:
df_voc['voc_day'] = df_voc['start_datetime'].astype('datetime64').dt.day
df_voc['voc_hour'] = df_voc['start_datetime'].astype('datetime64').dt.hour
# df_voc['voc_dayofweek'] = df_voc['start_datetime'].astype('datetime64').dt.dayofweek

In [20]:
phone_no_m = df_voc[['phone_no_m']].copy()
phone_no_m = phone_no_m.drop_duplicates(subset=['phone_no_m'], keep='last')

In [21]:
# 通话次数，通话人数
tmp = df_voc.groupby('phone_no_m')['opposite_no_m'].agg(opposite_cnt='count', opposite_nunique='nunique')
phone_no_m = phone_no_m.merge(tmp, how='left', on='phone_no_m')

# 人均通话次数
phone_no_m['voc_cnt_per_capita'] = phone_no_m['opposite_cnt'] / (phone_no_m['opposite_nunique'] + 0.0001)

del tmp
gc.collect()

0

In [22]:
"""
主叫通话
"""

df_calltype_id_1 = df_voc.loc[df_voc['calltype_id'] == 1, :].copy()

# 主叫通话次数，主叫通话使用的手机个数
tmp = df_calltype_id_1.groupby('phone_no_m')['imei_m'].agg(voc_calltype_id_1_cnt='count', imeis='nunique')
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')
del tmp
gc.collect()

# 主叫通话时长
tmp = df_calltype_id_1.groupby('phone_no_m')['call_dur'].agg(voc_calltype_id_1_call_dur_sum='sum')
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')
del tmp
gc.collect()

# 主叫通话时长小于30s的次数
tmp1 = df_calltype_id_1[df_calltype_id_1['call_dur'] < 30]
tmp2 = tmp1.groupby('phone_no_m')['call_dur'].agg(voc_calltype_id_1_30s_cnt='count')
phone_no_m = phone_no_m.merge(tmp2, on='phone_no_m', how='left')
del tmp1, tmp2
gc.collect()

# 主叫通话时长小于60s的次数
tmp1 = df_calltype_id_1[df_calltype_id_1['call_dur'] < 60]
tmp2 = tmp1.groupby('phone_no_m')['call_dur'].agg(voc_calltype_id_1_60s_cnt='count')
phone_no_m = phone_no_m.merge(tmp2, on='phone_no_m', how='left')
del tmp1, tmp2
gc.collect()

# 主叫通话时长大于300s（5分钟）的次数
tmp1 = df_calltype_id_1[df_calltype_id_1['call_dur'] > 300]
tmp2 = tmp1.groupby('phone_no_m')['call_dur'].agg(voc_calltype_id_1_300s_cnt='count')
phone_no_m = phone_no_m.merge(tmp2, on='phone_no_m', how='left')
del tmp1, tmp2
gc.collect()

# 主叫通话时长小于30s的次数的占比
phone_no_m['voc_calltype_id_1_30s_rate'] = phone_no_m['voc_calltype_id_1_30s_cnt'] / phone_no_m['voc_calltype_id_1_cnt']

# 主叫通话时长小于60s的次数的占比
phone_no_m['voc_calltype_id_1_60s_rate'] = phone_no_m['voc_calltype_id_1_60s_cnt'] / phone_no_m['voc_calltype_id_1_cnt']

# 主叫通话时长大于300s（5分钟）的次数的占比
phone_no_m['voc_calltype_id_1_300s_rate'] = phone_no_m['voc_calltype_id_1_300s_cnt'] / phone_no_m['voc_calltype_id_1_cnt']


# 主叫通话次数的占比
phone_no_m["call_type_id_1_rate"] = phone_no_m['voc_calltype_id_1_cnt'] / phone_no_m['opposite_cnt']

# 主叫通话时所在地市变动的个数
tmp = df_calltype_id_1.groupby('phone_no_m')['city_name'].agg(city_name_nunique='nunique')
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')
del tmp
gc.collect()

# 主叫通话时所在区县变动的个数
tmp = df_calltype_id_1.groupby("phone_no_m")['county_name'].agg(county_name_nunique='nunique')
phone_no_m = phone_no_m.merge(tmp, on="phone_no_m", how="left")
del tmp
gc.collect()

# 主叫通话时所在地市_区县变动的个数
tmp = df_calltype_id_1.groupby("phone_no_m")['city_name_county_name'].agg(city_name_county_name_nunique='nunique')
phone_no_m = phone_no_m.merge(tmp, on="phone_no_m", how="left")
del tmp
gc.collect()

0

In [ ]:
"""
被叫通话
"""

df_calltype_id_2 = df_voc.loc[df_voc['calltype_id'] == 2, :].copy()

# 主叫通话次数，主叫通话使用的手机个数
tmp = df_calltype_id_2.groupby('phone_no_m')['imei_m'].agg(voc_calltype_id_1_cnt='count', imeis='nunique')
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')
del tmp
gc.collect()

# 主叫通话时长
tmp = df_calltype_id_1.groupby('phone_no_m')['call_dur'].agg(voc_calltype_id_1_call_dur_sum='sum')
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')
del tmp
gc.collect()

# 主叫通话时长小于30s的次数
tmp1 = df_calltype_id_1[df_calltype_id_1['call_dur'] < 30]
tmp2 = tmp1.groupby('phone_no_m')['call_dur'].agg(voc_calltype_id_1_30s_cnt='count')
phone_no_m = phone_no_m.merge(tmp2, on='phone_no_m', how='left')
del tmp1, tmp2
gc.collect()

# 主叫通话时长小于60s的次数
tmp1 = df_calltype_id_1[df_calltype_id_1['call_dur'] < 60]
tmp2 = tmp1.groupby('phone_no_m')['call_dur'].agg(voc_calltype_id_1_60s_cnt='count')
phone_no_m = phone_no_m.merge(tmp2, on='phone_no_m', how='left')
del tmp1, tmp2
gc.collect()

# 主叫通话时长大于300s（5分钟）的次数
tmp1 = df_calltype_id_1[df_calltype_id_1['call_dur'] > 300]
tmp2 = tmp1.groupby('phone_no_m')['call_dur'].agg(voc_calltype_id_1_300s_cnt='count')
phone_no_m = phone_no_m.merge(tmp2, on='phone_no_m', how='left')
del tmp1, tmp2
gc.collect()

# 主叫通话时长小于30s的次数的占比
phone_no_m['voc_calltype_id_1_30s_rate'] = phone_no_m['voc_calltype_id_1_30s_cnt'] / phone_no_m['voc_calltype_id_1_cnt']

# 主叫通话时长小于60s的次数的占比
phone_no_m['voc_calltype_id_1_60s_rate'] = phone_no_m['voc_calltype_id_1_60s_cnt'] / phone_no_m['voc_calltype_id_1_cnt']

# 主叫通话时长大于300s（5分钟）的次数的占比
phone_no_m['voc_calltype_id_1_300s_rate'] = phone_no_m['voc_calltype_id_1_300s_cnt'] / phone_no_m['voc_calltype_id_1_cnt']


# 主叫通话次数的占比
phone_no_m["call_type_id_1_rate"] = phone_no_m['voc_calltype_id_1_cnt'] / phone_no_m['opposite_cnt']

# 主叫通话时所在地市变动的个数
tmp = df_calltype_id_1.groupby('phone_no_m')['city_name'].agg(city_name_nunique='nunique')
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')
del tmp
gc.collect()

# 主叫通话时所在区县变动的个数
tmp = df_calltype_id_1.groupby("phone_no_m")['county_name'].agg(county_name_nunique='nunique')
phone_no_m = phone_no_m.merge(tmp, on="phone_no_m", how="left")
del tmp
gc.collect()

# 主叫通话时所在地市_区县变动的个数
tmp = df_calltype_id_1.groupby("phone_no_m")['city_name_county_name'].agg(city_name_county_name_nunique='nunique')
phone_no_m = phone_no_m.merge(tmp, on="phone_no_m", how="left")
del tmp
gc.collect()

In [23]:
"""
被叫通话
"""

df_calltype_id_2 = df_voc.loc[df_voc['calltype_id'] == 2, :].copy()

# 被叫通话的次数
tmp = df_calltype_id_2.groupby('phone_no_m')['imei_m'].agg(voc_calltype_id_2_cnt='count')
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')

# 被叫通话次数的占比
phone_no_m["call_type_id_2_rate"] = phone_no_m['voc_calltype_id_2_cnt'] / phone_no_m['opposite_cnt']

del tmp, df_calltype_id_2
gc.collect()

20

In [24]:
"""
呼叫转移
"""

df_calltype_id_3 = df_voc.loc[df_voc['calltype_id'] == 3, :].copy()

# 呼叫转移的次数
tmp = df_calltype_id_3.groupby('phone_no_m')['imei_m'].agg(voc_calltype_id_3_cnt='count')
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')

# 呼叫转移次数的占比
phone_no_m["call_type_id_3_rate"] = phone_no_m['voc_calltype_id_3_cnt'] / phone_no_m['opposite_cnt']

del tmp, df_calltype_id_3
gc.collect()

20

In [25]:
"""
与对端通话统计
"""

# 与对端通话次数，与对端通话时长
tmp = df_voc.groupby(['phone_no_m', 'opposite_no_m'])['call_dur'].agg(call_count='count',
                                                                      call_sum='sum')

# 与对端通话次数的统计量
phone2opposite = tmp.groupby('phone_no_m')['call_count'].agg(phone2opposite_cnt_mean='mean',
                                                             phone2opposite_cnt_median='median',
                                                             phone2opposite_cnt_min='min',
                                                             phone2opposite_cnt_max='max',
                                                             phone2opposite_cnt_std='std')
phone_no_m = phone_no_m.merge(phone2opposite, on='phone_no_m', how='left')
del phone2opposite
gc.collect()


# 与对端总通话时长的统计量
phone2opposite = tmp.groupby('phone_no_m')['call_sum'].agg(phone2opposite_call_dur_mean='mean',
                                                           phone2opposite_call_dur_median='median',
                                                           phone2opposite_call_dur_min='min',
                                                           phone2opposite_call_dur_max='max',
                                                           phone2opposite_call_dur_std='std')
phone_no_m = phone_no_m.merge(phone2opposite, on='phone_no_m', how='left')
del phone2opposite, tmp
gc.collect()

0

In [26]:
"""
通话时长的统计
"""

# 通话时长的统计量
tmp = df_voc.groupby('phone_no_m')['call_dur'].agg(call_dur_mean='mean',
                                                   call_dur_median='median',
                                                   call_dur_max='max',
                                                   call_dur_min='min',
                                                   call_dur_std='std',
                                                   call_dur_sum='sum')
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')
del tmp
gc.collect()

0

In [27]:
"""
收费号码位置变动
"""

# 收费号码所在地市的个数
tmp = df_voc.groupby('phone_no_m')['city_name'].agg(city_name_nunique='nunique')
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')

# 收费号码所在区县的个数
tmp = df_voc.groupby('phone_no_m')['county_name'].agg(county_name_nunique='nunique')
phone_no_m = phone_no_m.merge(tmp, on="phone_no_m", how='left')

# 收费号码所在地市_区县的个数
tmp = df_voc.groupby('phone_no_m')['city_name_county_name'].agg(city_name_county_name_nunique='nunique')
phone_no_m = phone_no_m.merge(tmp, on="phone_no_m", how='left')

# 收费号码通话类型的个数
tmp = df_voc.groupby('phone_no_m')['calltype_id'].agg(calltype_id_unique='nunique')
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')

del tmp
gc.collect()

0

In [28]:
"""
通话时间点的偏好
"""

# hour通话次数最高
tmp = df_voc.groupby('phone_no_m')['voc_hour'].agg(voc_hour_mode=lambda x: stats.mode(x)[0][0],        # 频次最高的元素
                                                   voc_hour_mode_count=lambda x: stats.mode(x)[1][0],  # 频次最高的元素的频次
                                                   voc_hour_nunique='nunique')
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')

# day通话次数最高
tmp = df_voc.groupby('phone_no_m')['voc_day'].agg(voc_day_mode=lambda x: stats.mode(x)[0][0],          # 频次最高的元素
                                                  voc_day_mode_count=lambda x: stats.mode(x)[1][0],    # 频次最高的元素的频次
                                                  voc_day_nunique='nunique')
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')

del tmp
gc.collect()

0

In [29]:
# 每天的通话次数
voc_day_cnt_res = df_voc.groupby(['phone_no_m', 'voc_day'])['phone_no_m'].count().unstack()
for i in df_voc['voc_day'].unique():
    phone_no_m['voc_day{}_count'.format(i)] = phone_no_m['phone_no_m'].map(voc_day_cnt_res[i])

    
# 每天的通话人数
voc_day_nunique_res = df_voc.groupby(['phone_no_m', 'voc_day'])['opposite_no_m'].nunique().unstack()
for i in df_voc['voc_day'].unique():
    phone_no_m['voc_day{}_nunique'.format(i)] = phone_no_m['phone_no_m'].map(voc_day_nunique_res[i])

    
# 每天的通话时长
voc_day_call_dur_res = df_voc.groupby(['phone_no_m', 'voc_day'])['call_dur'].sum().unstack()
for i in df_voc['voc_day'].unique():
    phone_no_m['voc_day{}_call_dur_sum'.format(i)] = phone_no_m['phone_no_m'].map(voc_day_call_dur_res[i])



# 每小时的通话次数
voc_hour_cnt_res = df_voc.groupby(['phone_no_m', 'voc_hour'])['phone_no_m'].count().unstack()
for i in df_voc['voc_hour'].unique():
    phone_no_m['voc_hour{}_count'.format(i)] = phone_no_m['phone_no_m'].map(voc_hour_cnt_res[i])
    
# 每小时的通话人数
voc_hour_nunique_res = df_voc.groupby(['phone_no_m', 'voc_hour'])['opposite_no_m'].nunique().unstack()
for i in df_voc['voc_hour'].unique():
    phone_no_m['voc_hour{}_nunique'.format(i)] = phone_no_m['phone_no_m'].map(voc_hour_nunique_res[i])
    

# 每小时的通话时长
voc_hour_call_dur_res = df_voc.groupby(['phone_no_m', 'voc_hour'])['call_dur'].sum().unstack()
for i in df_voc['voc_hour'].unique():
    phone_no_m['voc_hour{}_call_dur_sum'.format(i)] = phone_no_m['phone_no_m'].map(voc_hour_call_dur_res[i])

In [30]:
phone_no_m.to_hdf('../input/voc_features.h5', 'df', index=False)
del phone_no_m
gc.collect()

75

## sms表

In [31]:
train_sms = pd.read_hdf('../input/train_sms.h5')
test_sms = pd.read_hdf('../input/test_sms.h5')

In [32]:
df_sms = pd.concat([train_sms, test_sms])

del train_sms, test_sms
gc.collect()

106

In [33]:
df_sms['sms_day'] = df_sms['request_datetime'].astype('datetime64').dt.day
df_sms['sms_hour'] = df_sms['request_datetime'].astype('datetime64').dt.hour
# df_sms['sms_dayofweek'] = df_sms['request_datetime'].astype('datetime64').dt.dayofweek

In [34]:
phone_no_m = df_sms[["phone_no_m"]].copy()
phone_no_m = phone_no_m.drop_duplicates(subset=['phone_no_m'], keep='last')

In [35]:
# 短信次数，短信人数
tmp = df_sms.groupby('phone_no_m')['opposite_no_m'].agg(sms_cnt='count', sms_nunique='nunique')

# 人均短信次数
tmp['sms_cnt_per_capita'] = tmp['sms_cnt'] / (tmp['sms_nunique'] + 0.0001)
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')

del tmp
gc.collect()

0

In [36]:
"""
短信上行，短信下行
"""

# 短信上行
df_sms_calltype1 = df_sms[df_sms['calltype_id'] == 1].copy()
tmp = df_sms_calltype1.groupby('phone_no_m')['calltype_id'].agg(sms_calltype1_cnt="count")
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')

# 短信上行比例
phone_no_m['sms_calltype1_rate'] = phone_no_m['sms_calltype1_cnt'] / phone_no_m['sms_cnt']

del tmp, df_sms_calltype1

# 短信下行
df_sms_calltype2 = df_sms[df_sms['calltype_id'] == 2].copy()
tmp = df_sms_calltype2.groupby('phone_no_m')['calltype_id'].agg(sms_calltype2_cnt="count")
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')

# 短信上行/短信下行
phone_no_m['type1_rate/type2_rate'] = phone_no_m['sms_calltype1_cnt'] / (phone_no_m['sms_calltype2_cnt'] + 0.00001)

# 删除，效果变差
# # 短信下行/短信上行
# phone_no_m['type2_rate/type1_rate'] = phone_no_m['sms_calltype2_cnt'] / (phone_no_m['sms_calltype1_cnt'] + 0.00001)

del tmp, df_sms_calltype2
gc.collect()

0

In [37]:
"""
短信时间点的偏好
"""

tmp = df_sms.groupby('phone_no_m')['sms_hour'].agg(sms_hour_mode=lambda x: stats.mode(x)[0][0],        # 频次最高的元素
                                                   sms_hour_mode_count=lambda x: stats.mode(x)[1][0],  # 频次最高的元素的频次
                                                   sms_hour_nunique='nunique')
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')

del tmp

tmp = df_sms.groupby('phone_no_m')['sms_day'].agg(sms_day_mode=lambda x: stats.mode(x)[0][0],
                                                  sms_day_mode_count=lambda x: stats.mode(x)[1][0],
                                                  sms_day_nunique='nunique')
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')

del tmp
gc.collect()

0

In [38]:
# 每天的短信次数
sms_day_res = df_sms.groupby(['phone_no_m', 'sms_day'])['phone_no_m'].count().unstack()
for i in df_sms['sms_day'].unique():
    phone_no_m['sms_day{}_count'.format(i)] = phone_no_m['phone_no_m'].map(sms_day_res[i])


# 每小时的短信次数
sms_hour_res = df_sms.groupby(['phone_no_m', 'sms_hour'])['phone_no_m'].count().unstack()
for i in df_sms['sms_hour'].unique():
    phone_no_m['sms_hour{}_count'.format(i)] = phone_no_m['phone_no_m'].map(sms_hour_res[i])

In [39]:
phone_no_m.to_hdf('../input/sms_features.h5', 'df', index=False)

del phone_no_m
gc.collect()

64

## app表

In [40]:
train_app = pd.read_hdf('../input/train_app.h5')
test_app = pd.read_hdf('../input/test_app.h5')

In [41]:
df_app = pd.concat([train_app, test_app])

del train_app, test_app
gc.collect()

106

In [42]:
phone_no_m = df_app[['phone_no_m']].copy()
phone_no_m = phone_no_m.drop_duplicates(subset=['phone_no_m'], keep='last')

In [43]:
# APP数
tmp = df_app.groupby('phone_no_m')['busi_name'].agg(busi_cnt='count')
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')
del tmp
gc.collect()

40

In [44]:
"""
流量统计
"""
tmp = df_app.groupby("phone_no_m")["flow"].agg(flow_mean='mean',
                                               flow_median='median',
                                               flow_min='min',
                                               flow_max='max',
                                               flow_std='std',
                                               flow_sum='sum',
                                               flow_skew='skew')
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')

del tmp
gc.collect()

20

In [45]:
phone_no_m.to_hdf('../input/app_features.h5', 'df', index=False)

del phone_no_m
gc.collect()

75

## 读取数据，建模

In [46]:
df_user = reduce_mem_usage(pd.read_hdf('../input/user_features.h5'))
df_voc = reduce_mem_usage(pd.read_hdf('../input/voc_features.h5'))
df_sms = reduce_mem_usage(pd.read_hdf('../input/sms_features.h5'))
# df_app = reduce_mem_usage(pd.read_hdf('../input/app_features.h5'))

Memory usage of dataframe is 3651648.00 MB
Memory usage after optimization is: 1760616.00 MB
Decreased by 51.8%
Memory usage of dataframe is 11485296.00 MB
Memory usage after optimization is: 2979932.00 MB
Decreased by 74.1%
Memory usage of dataframe is 3850000.00 MB
Memory usage after optimization is: 1031250.00 MB
Decreased by 73.2%


In [47]:
df_user.shape, df_voc.shape, df_sms.shape #, df_app.shape

((8151, 85), (6788, 212), (6875, 69))

In [48]:
df = df_user.merge(df_voc, on='phone_no_m', how='left')
df = df.merge(df_sms, on='phone_no_m', how='left')
# df = df.merge(df_app, on='phone_no_m', how='left')

del df_user, df_voc, df_sms#, df_app
gc.collect()

0

In [49]:
df.shape

(8151, 364)

In [50]:
# astype('category')会掉分，不做此处理
# for i in tqdm(cat_feat):
#     print(i)
#     df[i] = df[i].astype('category')

100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 601.42it/s]

city_name
county_name
city_name_county_name


In [51]:
df_train = df[df.label.notna()]
df_test = df[df.label.isna()]

df_train.shape, df_test.shape

((6106, 364), (2045, 364))

In [52]:
fs = FeatureSelector(data=df_train.drop(['phone_no_m', 'label'], axis=1), labels=df_train['label'])

fs.identify_all(selection_params={'missing_threshold': 0.98,
                                  'correlation_threshold': 0.98, 
                                  'task': 'classification',
                                  'eval_metric': 'auc', 
                                  'cumulative_importance': 0.99})

6 features with greater than 0.98 missing values.

0 features with a single unique value.

45 features with a correlation magnitude greater than 0.98.

Training Gradient Boosting Model

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[99]	valid_0's auc: 0.952042	valid_0's binary_logloss: 0.20526
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[105]	valid_0's auc: 0.954754	valid_0's binary_logloss: 0.190864
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[97]	valid_0's auc: 0.953382	valid_0's binary_logloss: 0.199934
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[42]	valid_0's auc: 0.968212	valid_0's binary_logloss: 0.208802
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[90]	valid_0's auc: 0.957631	valid_0's binary_logloss: 0.194596
Training 

In [53]:
train_removed_all_once = fs.remove(methods='all')
# train_removed_all_once

['missing', 'single_unique', 'collinear', 'zero_importance', 'low_importance'] methods have been run

Removed 484 features.


In [54]:
use_cols = train_removed_all_once.columns.to_list()

In [55]:
X_train, X_valid, y_train, y_valid = train_test_split(df_train[use_cols], df_train['label'],
                                                      test_size=0.2,
                                                      random_state=2020)

In [56]:
train_cols = [i for i in X_train.columns if i not in ['phone_no_m', 'label']]

In [57]:
params = {'objective': 'binary',
          'boosting': 'gbdt',
          'metric': 'auc',
          'learning_rate': 0.1,
          'num_leaves': 31,
          'lambda_l1': 0.1,
          'lambda_l2': 0,
          'min_data_in_leaf': 20,
          'is_unbalance': True,
          'max_depth': -1,
          'seed': 2020}

In [58]:
useful_cols = []
useless_cols = []

for i in train_cols:
    print(i)
    
    lgb_train = lgb.Dataset(X_train[[i]].values, y_train) 
    lgb_valid= lgb.Dataset(X_valid[[i]].values, y_valid, reference=lgb_train)
    lgb_test = lgb.train(params,
                         lgb_train,
                         num_boost_round=1000,
                         valid_sets=[lgb_valid, lgb_train],
                         early_stopping_rounds=50,
                         verbose_eval=20)
    
    print('*' * 5)
    print(lgb_test.best_score['valid_0']['auc'])
    if lgb_test.best_score['valid_0']['auc'] > 0.52:
        useful_cols.append(i)
    else:
        useless_cols.append(i)
    print('*' * 20)
    print('\n')

city_name
Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.684155	valid_0's auc: 0.671655
[40]	training's auc: 0.684156	valid_0's auc: 0.672408
[60]	training's auc: 0.684156	valid_0's auc: 0.672408
[80]	training's auc: 0.684158	valid_0's auc: 0.671655
Early stopping, best iteration is:
[37]	training's auc: 0.684156	valid_0's auc: 0.672408
*****
0.6724078657865786
********************


county_name
Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.767358	valid_0's auc: 0.72344
[40]	training's auc: 0.772283	valid_0's auc: 0.728765
[60]	training's auc: 0.77374	valid_0's auc: 0.730307
[80]	training's auc: 0.774604	valid_0's auc: 0.730896
[100]	training's auc: 0.775263	valid_0's auc: 0.73182
[120]	training's auc: 0.775831	valid_0's auc: 0.731191
[140]	training's auc: 0.776325	valid_0's auc: 0.732008
Early stopping, best iteration is:
[90]	training's auc: 0.775065	valid_0's auc: 0.732017
*****
0.7320170875783231
***********

Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.666928	valid_0's auc: 0.663023
[40]	training's auc: 0.666928	valid_0's auc: 0.663023
Early stopping, best iteration is:
[1]	training's auc: 0.666928	valid_0's auc: 0.663023
*****
0.6630225522552256
********************


county_name_arpu_202004_max
Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.767023	valid_0's auc: 0.726908
[40]	training's auc: 0.771569	valid_0's auc: 0.72833
[60]	training's auc: 0.773169	valid_0's auc: 0.729605
[80]	training's auc: 0.77389	valid_0's auc: 0.73028
[100]	training's auc: 0.774547	valid_0's auc: 0.730086
[120]	training's auc: 0.774957	valid_0's auc: 0.730044
[140]	training's auc: 0.775215	valid_0's auc: 0.729886
Early stopping, best iteration is:
[101]	training's auc: 0.774606	valid_0's auc: 0.730427
*****
0.7304267111493759
********************


county_name_arpu_202004_median
Training until validation scores don't improve for 50 round

[200]	training's auc: 0.761222	valid_0's auc: 0.715495
[220]	training's auc: 0.761359	valid_0's auc: 0.715626
[240]	training's auc: 0.761477	valid_0's auc: 0.715994
[260]	training's auc: 0.761644	valid_0's auc: 0.716388
[280]	training's auc: 0.761759	valid_0's auc: 0.7164
[300]	training's auc: 0.761881	valid_0's auc: 0.71655
[320]	training's auc: 0.761925	valid_0's auc: 0.71663
[340]	training's auc: 0.762033	valid_0's auc: 0.716625
[360]	training's auc: 0.762077	valid_0's auc: 0.716699
[380]	training's auc: 0.762127	valid_0's auc: 0.717013
[400]	training's auc: 0.762305	valid_0's auc: 0.717551
[420]	training's auc: 0.762329	valid_0's auc: 0.717566
[440]	training's auc: 0.762337	valid_0's auc: 0.717584
[460]	training's auc: 0.762357	valid_0's auc: 0.717551
[480]	training's auc: 0.762391	valid_0's auc: 0.717626
[500]	training's auc: 0.762459	valid_0's auc: 0.717605
[520]	training's auc: 0.762466	valid_0's auc: 0.717662
[540]	training's auc: 0.76247	valid_0's auc: 0.717566
[560]	training'

Early stopping, best iteration is:
[46]	training's auc: 0.862802	valid_0's auc: 0.802781
*****
0.802781365093031
********************


call_dur_sum
Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.878134	valid_0's auc: 0.834136
[40]	training's auc: 0.881425	valid_0's auc: 0.835308
[60]	training's auc: 0.883362	valid_0's auc: 0.835528
[80]	training's auc: 0.884713	valid_0's auc: 0.836432
[100]	training's auc: 0.885712	valid_0's auc: 0.83603
[120]	training's auc: 0.886397	valid_0's auc: 0.836305
Early stopping, best iteration is:
[89]	training's auc: 0.885281	valid_0's auc: 0.836628
*****
0.8366276845075812
********************


county_name_nunique_y
Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.834337	valid_0's auc: 0.811923
[40]	training's auc: 0.83441	valid_0's auc: 0.811185
[60]	training's auc: 0.834467	valid_0's auc: 0.811551
Early stopping, best iteration is:
[17]	training's auc: 0.834214	valid_0's auc: 0.81

[20]	training's auc: 0.761157	valid_0's auc: 0.737866
[40]	training's auc: 0.761484	valid_0's auc: 0.735977
Early stopping, best iteration is:
[7]	training's auc: 0.760881	valid_0's auc: 0.738262
*****
0.738262005548381
********************


voc_day7_count
Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.74877	valid_0's auc: 0.739713
[40]	training's auc: 0.748915	valid_0's auc: 0.741171
[60]	training's auc: 0.74908	valid_0's auc: 0.741299
[80]	training's auc: 0.749142	valid_0's auc: 0.741407
[100]	training's auc: 0.74942	valid_0's auc: 0.741332
[120]	training's auc: 0.749432	valid_0's auc: 0.741371
Early stopping, best iteration is:
[84]	training's auc: 0.749143	valid_0's auc: 0.741443
*****
0.7414427584062754
********************


voc_day14_count
Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.75402	valid_0's auc: 0.744502
[40]	training's auc: 0.754056	valid_0's auc: 0.743837
[60]	training's auc: 0.754073	valid_0

...

[40]	training's auc: 0.793771	valid_0's auc: 0.794088
Early stopping, best iteration is:
[1]	training's auc: 0.79327	valid_0's auc: 0.794456
*****
0.7944558042760798
********************


voc_hour11_nunique
Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.816171	valid_0's auc: 0.789437
[40]	training's auc: 0.816409	valid_0's auc: 0.789863
[60]	training's auc: 0.816545	valid_0's auc: 0.789725
[80]	training's auc: 0.816888	valid_0's auc: 0.78925
[100]	training's auc: 0.817125	valid_0's auc: 0.789414
Early stopping, best iteration is:
[58]	training's auc: 0.81654	valid_0's auc: 0.789997
*****
0.7899970703592099
********************


voc_hour9_nunique
Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.793032	valid_0's auc: 0.765709
[40]	training's auc: 0.79367	valid_0's auc: 0.765529
[60]	training's auc: 0.793723	valid_0's auc: 0.765246
Early stopping, best iteration is:
[26]	training's auc: 0.79324	valid_0's auc: 0.7669

[60]	training's auc: 0.791557	valid_0's auc: 0.744501
Early stopping, best iteration is:
[13]	training's auc: 0.785811	valid_0's auc: 0.74816
*****
0.7481600062180132
********************


voc_hour10_call_dur_sum
Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.808841	valid_0's auc: 0.788585
[40]	training's auc: 0.811849	valid_0's auc: 0.787885
[60]	training's auc: 0.814009	valid_0's auc: 0.786724
Early stopping, best iteration is:
[12]	training's auc: 0.806013	valid_0's auc: 0.78922
*****
0.7892198187210026
********************


voc_hour13_call_dur_sum
Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.805556	valid_0's auc: 0.781108
[40]	training's auc: 0.808302	valid_0's auc: 0.779414
[60]	training's auc: 0.80977	valid_0's auc: 0.780479
Early stopping, best iteration is:
[12]	training's auc: 0.804395	valid_0's auc: 0.782775
*****
0.7827746089826374
********************


voc_hour22_call_dur_sum
Training until valid

[40]	training's auc: 0.901742	valid_0's auc: 0.867266
Early stopping, best iteration is:
[7]	training's auc: 0.901688	valid_0's auc: 0.868603
*****
0.8686026211316784
********************


sms_day1_count
Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.815883	valid_0's auc: 0.785885
[40]	training's auc: 0.8159	valid_0's auc: 0.786227
[60]	training's auc: 0.815935	valid_0's auc: 0.785991
[80]	training's auc: 0.815932	valid_0's auc: 0.786171
Early stopping, best iteration is:
[45]	training's auc: 0.815899	valid_0's auc: 0.78641
*****
0.7864097551059454
********************


sms_day2_count
Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.810768	valid_0's auc: 0.786628
[40]	training's auc: 0.811138	valid_0's auc: 0.786658
Early stopping, best iteration is:
[3]	training's auc: 0.810721	valid_0's auc: 0.787444
*****
0.7874440976706366
********************


sms_day3_count
Training until validation scores don't improve fo

Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.735239	valid_0's auc: 0.695329
[40]	training's auc: 0.735248	valid_0's auc: 0.695328
[60]	training's auc: 0.735251	valid_0's auc: 0.695328
Early stopping, best iteration is:
[27]	training's auc: 0.735246	valid_0's auc: 0.69537
*****
0.6953696728368489
********************


sms_day23_count
Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.773557	valid_0's auc: 0.754412
[40]	training's auc: 0.774245	valid_0's auc: 0.754661
[60]	training's auc: 0.774399	valid_0's auc: 0.754672
Early stopping, best iteration is:
[25]	training's auc: 0.774084	valid_0's auc: 0.755384
*****
0.7553839623092744
********************


sms_day24_count
Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.871818	valid_0's auc: 0.847497
[40]	training's auc: 0.871907	valid_0's auc: 0.846426
Early stopping, best iteration is:
[1]	training's auc: 0.870984	valid_0's auc: 0

[120]	training's auc: 0.762755	valid_0's auc: 0.734857
Early stopping, best iteration is:
[79]	training's auc: 0.762636	valid_0's auc: 0.73489
*****
0.7348899292103124
********************


sms_hour15_count
Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.831972	valid_0's auc: 0.814301
[40]	training's auc: 0.832531	valid_0's auc: 0.813359
[60]	training's auc: 0.832827	valid_0's auc: 0.811712
Early stopping, best iteration is:
[23]	training's auc: 0.832027	valid_0's auc: 0.814395
*****
0.8143952982254747
********************


sms_hour2_count
Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.531729	valid_0's auc: 0.522104
[40]	training's auc: 0.531752	valid_0's auc: 0.522116
Early stopping, best iteration is:
[1]	training's auc: 0.530025	valid_0's auc: 0.52276
*****
0.5227600205672741
********************


sms_hour12_count
Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.83786	valid

In [60]:
print(useless_cols)

['voc_hour3_call_dur_sum']


In [61]:
lgb_train = lgb.Dataset(X_train[useful_cols].values, y_train) 

lgb_valid= lgb.Dataset(X_valid[useful_cols].values, y_valid, reference=lgb_train)  

print('Start training...')

lgb_val_0 = lgb.train(params,
                      lgb_train,
                      num_boost_round=10000,
                      valid_sets=[lgb_valid, lgb_train],
                      early_stopping_rounds=100,
                      verbose_eval=10)

print('Done!')

Start training...
Training until validation scores don't improve for 100 rounds
[10]	training's auc: 0.988563	valid_0's auc: 0.948032
[20]	training's auc: 0.996447	valid_0's auc: 0.95581
[30]	training's auc: 0.998739	valid_0's auc: 0.956286
[40]	training's auc: 0.999479	valid_0's auc: 0.955194
[50]	training's auc: 0.999663	valid_0's auc: 0.952976
[60]	training's auc: 0.999769	valid_0's auc: 0.953129
[70]	training's auc: 0.999908	valid_0's auc: 0.952692
[80]	training's auc: 0.99998	valid_0's auc: 0.952902
[90]	training's auc: 0.999999	valid_0's auc: 0.953156
[100]	training's auc: 1	valid_0's auc: 0.952411
[110]	training's auc: 1	valid_0's auc: 0.951957
[120]	training's auc: 1	valid_0's auc: 0.952531
Early stopping, best iteration is:
[24]	training's auc: 0.997832	valid_0's auc: 0.957111
Done!


In [62]:
# 验证集结果
X_valid['prob'] = lgb_val_0.predict(X_valid[useful_cols])
X_valid['pred'] = np.where(X_valid['prob'] > 0.4735, 1, 0)

f1_04735 = np.round(f1_score(y_valid, X_valid['pred']), 4)
auc_04735 = roc_auc_score(y_valid, X_valid['prob'])

print('f1_04735: ', f1_04735)
print('auc_04735: ', auc_04735)

f1_04735:  0.874
auc_04735:  0.9571106567178457


In [63]:
lgb_train_all = lgb.Dataset(df_train[useful_cols].values, df_train['label'])   

print('Start training...')

lgb_model = lgb.train(params,
                      lgb_train_all,
                      num_boost_round=lgb_val_0.best_iteration + 20)

print('Done!')

Start training...
Done!


In [64]:
df_test['label'] = np.where(lgb_model.predict(df_test[useful_cols]) > 0.4735, 1, 0)
df_test[['phone_no_m', 'label']].to_csv('../sub/sub_{}_{}.csv'.format(time.strftime('%Y%m%d'), f1_04735), index=False)